In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import grasptext
from grasptext import GrASP, CustomAttribute, remove_specialized_patterns
from sklearn.model_selection import train_test_split
from typing import Iterable, List, Set, Callable, Optional, Union, Sequence

## Load the data
- We use the **IBM Debater® - Evidence Sentences** dataset from the following paper which can be downloaded [here](https://www.research.ibm.com/haifa/dept/vst/debating_data.shtml#Argument%20Detection). (Please select the ACL 2018 dataset)
```
Shnarch, E., Alzate, C., Dankin, L., Gleize, M., Hou, Y., Choshen, L., ... & Slonim, N. (2018, July). Will it blend? blending weak and strong labeled data in a neural network for argumentation mining. In Proceedings of the 56th Annual Meeting of the Association for Computational Linguistics (Volume 2: Short Papers) (pp. 599-605).
```

- Load the data

In [3]:
import pandas as pd
def get_data(split = 'train'):
    df = pd.read_csv(f'data/IBMDebaterEvidenceSentences/{split}.csv')
    texts = df['candidate masked'].tolist()
    labels = list(map(int, df['label'].tolist()))
    return texts, labels

In [4]:
X_train, y_train = get_data(split = 'train')
positive = [t for idx, t in enumerate(X_train) if y_train[idx]]
negative = [t for idx, t in enumerate(X_train) if not y_train[idx]]
print(len(positive), len(negative))

1499 2566


## (Optional) Add a custom attribute
This attribute checks whether a word is in a given list of argumentative words or not.

In [5]:
try:
    ARGUMENTATIVE_LEXICON = [line.strip().lower() for line in open('data/argumentative_unigrams_lexicon_shortlist.txt', 'r') if line.strip() != '']
    def _argumentative_extraction(text: str, tokens: List[str]) -> List[Set[str]]:
        tokens = map(str.lower, tokens)
        ans = []
        for t in tokens:
            t_ans = []
            if t.lower() in ARGUMENTATIVE_LEXICON:
                t_ans.append('Yes')
            ans.append(set(t_ans))
        return ans

    def _argumentative_translation(attr:str, 
                          is_complement:bool = False) -> str:
        word = attr.split(':')[1]
        assert word == 'Yes'
        return 'an argumentative word'

    ArgumentativeAttribute = CustomAttribute(name = 'ARGUMENTATIVE', extraction_function = _argumentative_extraction, translation_function = _argumentative_translation)

except:
    ArgumentativeAttribute = None

print(ArgumentativeAttribute)

ARGUMENTATIVE


## Run GrASP

In [6]:
# Create the GrASP engine
if ArgumentativeAttribute is not None:
    grasp_model = GrASP(gaps_allowed = 2, num_patterns = 100, include_standard = ['LEMMA', 'POS', 'NER', 'HYPERNM', 'SENTIMENT'],
                        include_custom = [ArgumentativeAttribute],
                        correlation_threshold = 0.5, alphabet_size = 100)
else:
    grasp_model = GrASP(gaps_allowed = 2, num_patterns = 100, include_standard = ['LEMMA', 'POS', 'NER', 'HYPERNM', 'SENTIMENT'],
                        correlation_threshold = 0.5, alphabet_size = 100)

In [7]:
# Fit GrASP to the dataset
the_patterns = grasp_model.fit_transform(positive, negative)

  0%|          | 5/1499 [00:00<00:34, 42.98it/s]

Step 1: Create augmented texts


100%|██████████| 2566/2566 [00:59<00:00, 43.33it/s]


Step 2: Find frequent attributes


  1%|          | 6/727 [00:00<00:16, 43.47it/s]

...

Length 3 / 5; New candidates = 14399
Finding top k: 10 / 100
Finding top k: 20 / 100
Finding top k: 30 / 100
Finding top k: 40 / 100
Finding top k: 50 / 100
Finding top k: 60 / 100
Finding top k: 70 / 100
Finding top k: 80 / 100
Finding top k: 90 / 100
Finding top k: 100 / 100
Example of current patterns
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: During an October 2004 debate , Obama stated that:['LEMMA:that', 'SPACY:POS-ADP'] he opposed TOPIC_CONCEPT for use at private schools because he believes they would undermine public schools [ REF ] . 
-------------------------
[MATCH]: Harvard Director Edward Green concluded that:['LEMMA:that', 'SPACY:POS-ADP'] the " best studies " show that more TOPIC_CONCEPT promote the spread of AIDS [ REF ] . 
-------------------------
Counterexamples ~ Not class Positive:
[MATCH]: Although TOPIC_CONCEPT is illegal , 

100%|██████████| 32/32 [00:42<00:00,  1.34s/it]


Length 4 / 5; New candidates = 6348
Finding top k: 10 / 100
Finding top k: 20 / 100
Finding top k: 30 / 100
Finding top k: 40 / 100
Finding top k: 50 / 100
Finding top k: 60 / 100
Finding top k: 70 / 100
Finding top k: 80 / 100
Finding top k: 90 / 100
Finding top k: 100 / 100
Example of current patterns
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: For example , a review of the evidence by Louise Arsenault , et al . , in 2004 reports that:['LEMMA:that', 'SPACY:POS-ADP'] on an individual level , TOPIC_CONCEPT use confers an overall twofold increase in the relative risk of later schizophrenia , assuming a causal relationship . 
-------------------------
[MATCH]: In an interview on Larry King Live , Weil claimed that:['LEMMA:that', 'SPACY:POS-ADP'] sugar , starch , refined carbohydrates , and TOPIC_CONCEPT are more dangerous to the human body than satur

100%|██████████| 10/10 [00:10<00:00,  1.00s/it]


Length 5 / 5; New candidates = 1987
Finding top k: 10 / 100
Finding top k: 20 / 100
Finding top k: 30 / 100
Finding top k: 40 / 100
Finding top k: 50 / 100
Finding top k: 60 / 100
Finding top k: 70 / 100
Finding top k: 80 / 100
Finding top k: 90 / 100
Finding top k: 100 / 100
Example of current patterns
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: A report compiled by the Lanzhou University in 2006 warned that:['LEMMA:that', 'SPACY:POS-ADP'] the felling of the forests around Zhouqu for TOPIC_CONCEPT and agriculture would cause heavy soil erosion and destabilise hillsides . 
-------------------------
[MATCH]: German engineering company Siemens announced in July 2010 , following a visit to Russia by Chancellor Angela Merkel , that:['LEMMA:that', 'SPACY:POS-ADP'] it would build TOPIC_CONCEPT in Russia [ REF ] . 
-------------------------
Counterexample

In [10]:
# Print the learned patterns
for idx, p in enumerate(the_patterns):
    print(f'Rank {idx+1}')
    print(p)

Rank 1
Pattern: [['LEMMA:that', 'SPACY:POS-ADP']]
Window size: 3
Class: Positive
Precision: 0.546
Match: 1239 (30.5%)
Gain = 0.042
Metric (global) = 0.042
Examples ~ Class Positive:
[MATCH]: Zeigler believed Riley should be disciplined to say " Sir " and " Ma'am " to adults , and that:['SPACY:POS-ADP', 'LEMMA:that'] TOPIC_CONCEPT was the best means of disciplining a child . 
-------------------------
[MATCH]: Researchers found that:['SPACY:POS-ADP', 'LEMMA:that'] a removal of TOPIC_CONCEPT policies from all colleges and universities would result in a significant drop in minority presence in all institutions of higher education , upholding TOPIC_CONCEPT role in diversity in the American classroom [ REF ] . 
-------------------------
Counterexamples ~ Not class Positive:
[MATCH]: In a column published in the New York Times on June 15 , 2011 Kristof argued that:['SPACY:POS-ADP', 'LEMMA:that'] the United States military was a prime example of how comprehensive social safety net , TOPIC_CON

In [12]:
print(f'  #    class Cov(%)    Prec    Gain    Pattern')
for idx, p in enumerate(the_patterns):
    print(f'{idx+1:>3} {p.support_class}   {round(p.coverage*100, 1):>4}   {p.precision:.3f}   {p.metric:.3f}    {p.get_pattern_id()}')

  #    class Cov(%)    Prec    Gain    Pattern
  1 Positive   30.5   0.546   0.042    [['LEMMA:that', 'SPACY:POS-ADP']]
  2 Negative   42.7   0.546   0.017    [['LEMMA:]']]
  3 Negative   27.7   0.518   0.015    [['LEMMA:that'], ['SPACY:POS-VERB']]
  4 Positive    8.9   0.579   0.013    [['ARGUMENTATIVE:Yes'], ['LEMMA:that']]
  5 Negative   26.6   0.529   0.012    [['SPACY:POS-VERB'], ['SPACY:POS-ADP'], ['SPACY:POS-VERB']]
  6 Positive    5.2   0.627   0.011    [['LEMMA:study']]
  7 Positive    2.5   0.728   0.010    [['LEMMA:risk']]
  8 Negative   27.6   0.541   0.009    [['ARGUMENTATIVE:Yes'], ['SPACY:POS-ADP']]
  9 Positive    4.9   0.607   0.009    [['LEMMA:ban']]
 10 Positive    4.8   0.607   0.009    [['ARGUMENTATIVE:Yes'], ['ARGUMENTATIVE:Yes']]
 11 Positive   10.5   0.525   0.009    [['LEMMA:that', 'SPACY:POS-ADP'], ['SPACY:POS-DET']]
 12 Positive    6.0   0.578   0.008    [['LEMMA:energy']]
 13 Positive    7.7   0.546   0.008    [['ARGUMENTATIVE:Yes'], ['SPACY:POS-ADP'], ['SPA

## Save the patterns to a json file
We can use this json file as an input of the web demo tool for exploring the learned patterns and the training data

In [13]:
grasp_model.to_json('results/case_study_2.json')

100%|██████████| 100/100 [00:00<00:00, 231.03it/s]


Successfully dump the results to results/case_study_2.json
